In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

In [2]:
train_activations = np.load('/Volumes/Extreme SSD/progetto_tesi/improving_dementia_detection_model/explainability-dementia-alfio/local/detailed_inference/inference_20250512_102807/train_activations.npy', allow_pickle = True,).item()
test_activations = np.load('/Volumes/Extreme SSD/progetto_tesi/improving_dementia_detection_model/explainability-dementia-alfio/local/detailed_inference/inference_20250512_102807/test_activations.npy', allow_pickle = True,).item()
val_activations = np.load('/Volumes/Extreme SSD/progetto_tesi/improving_dementia_detection_model/explainability-dementia-alfio/local/detailed_inference/inference_20250512_102807/val_activations.npy', allow_pickle = True,).item()


In [3]:
for dataset_name, dataset in zip(['train_activations', 'test_activations', 'val_activations'], [train_activations, test_activations, val_activations]):
    list_of_data = [{'crop_file': key, 'valore': value} for key, value in dataset.items()]
    df = pd.DataFrame(list_of_data)
    globals()[f'{dataset_name}_df'] = df

In [4]:
train_activations_df['dataset'] = 'training'
test_activations_df['dataset'] = 'test'
val_activations_df['dataset'] = 'validation'

In [5]:
for dataset_name, dataset in zip(['train_activations_df', 'test_activations_df', 'val_activations_df'], [train_activations_df, test_activations_df, val_activations_df]):
    globals()[f'{dataset_name}']['valore_softmax'] = globals()[f'{dataset_name}']['valore'].apply(lambda x: softmax(x))
    globals()[f'{dataset_name}']['pred_label'] = globals()[f'{dataset_name}']['valore_softmax'].apply(lambda x: np.argmax(x))

In [6]:
all_activations_df = pd.concat([train_activations_df,test_activations_df,val_activations_df], ignore_index = True)

In [7]:
all_activations_df

,crop_file,valore,dataset,valore_softmax,pred_label
0,00000848.npy,"[0.9925883, -1.4026834, -1.0476341]",training,"[0.8189018, 0.074641176, 0.106457055]",0
1,00000849.npy,"[0.7074843, -0.79796463, -0.32676435]",training,"[0.63395023, 0.14068474, 0.2253651]",0
2,00000850.npy,"[0.76318306, -1.4512811, -0.8097436]",training,"[0.7595037, 0.082946934, 0.15754934]",0
3,00000851.npy,"[0.95496655, -1.6753643, -1.1246376]",training,"[0.8353979, 0.060194284, 0.10440776]",0
4,00000852.npy,"[0.7671418, -1.0445702, -0.53408056]",training,"[0.6965859, 0.113804154, 0.18960992]",0
...,...,...,...,...,...
69789,00069084.npy,"[-2.868002, -1.6037244, -0.08444479]",validation,"[0.04826955, 0.17089997, 0.7808305]",2
69790,00069085.npy,"[-1.6224732, -0.7622117, -0.14632873]",validation,"[0.12920196, 0.30540487, 0.56539315]",2
69791,00069086.npy,"[-0.78004813, -0.2351872, -0.5589846]",validation,"[0.25177684, 0.43415606, 0.31406707]",1
69792,00069087.npy,"[-0.6792966, -0.35036835, -0.4043098]",validation,"[0.2698333, 0.37492752, 0.35523918]",1


In [8]:
annot = pd.read_csv('/Users/alfioleanza/progetto_tesi/dataset-eeg/miltiadous_deriv_uV_d1.0s_o0.0s/annot_all_hc-ftd-ad.csv')

In [9]:
annot

,crop_file,label,original_rec,crop_start_sample,crop_end_sample
0,00000000.npy,0,sub-064,0,499.0
1,00000001.npy,0,sub-064,500,999.0
2,00000002.npy,0,sub-064,1000,1499.0
3,00000003.npy,0,sub-064,1500,1999.0
4,00000004.npy,0,sub-064,2000,2499.0
...,...,...,...,...,...
69789,00069789.npy,2,sub-033,350000,350499.0
69790,00069790.npy,2,sub-033,350500,350999.0
69791,00069791.npy,2,sub-033,351000,351499.0
69792,00069792.npy,2,sub-033,351500,351999.0


In [10]:
annot = annot.rename(columns={'label':'true_label'})

In [11]:
true_pred = all_activations_df.merge(annot, on = 'crop_file')

In [12]:
true_pred

,crop_file,valore,dataset,valore_softmax,pred_label,true_label,original_rec,crop_start_sample,crop_end_sample
0,00000848.npy,"[0.9925883, -1.4026834, -1.0476341]",training,"[0.8189018, 0.074641176, 0.106457055]",0,0,sub-037,0,499.0
1,00000849.npy,"[0.7074843, -0.79796463, -0.32676435]",training,"[0.63395023, 0.14068474, 0.2253651]",0,0,sub-037,500,999.0
2,00000850.npy,"[0.76318306, -1.4512811, -0.8097436]",training,"[0.7595037, 0.082946934, 0.15754934]",0,0,sub-037,1000,1499.0
3,00000851.npy,"[0.95496655, -1.6753643, -1.1246376]",training,"[0.8353979, 0.060194284, 0.10440776]",0,0,sub-037,1500,1999.0
4,00000852.npy,"[0.7671418, -1.0445702, -0.53408056]",training,"[0.6965859, 0.113804154, 0.18960992]",0,0,sub-037,2000,2499.0
...,...,...,...,...,...,...,...,...,...
69789,00069084.npy,"[-2.868002, -1.6037244, -0.08444479]",validation,"[0.04826955, 0.17089997, 0.7808305]",2,2,sub-026,446500,446999.0
69790,00069085.npy,"[-1.6224732, -0.7622117, -0.14632873]",validation,"[0.12920196, 0.30540487, 0.56539315]",2,2,sub-026,447000,447499.0
69791,00069086.npy,"[-0.78004813, -0.2351872, -0.5589846]",validation,"[0.25177684, 0.43415606, 0.31406707]",1,2,sub-026,447500,447999.0
69792,00069087.npy,"[-0.6792966, -0.35036835, -0.4043098]",validation,"[0.2698333, 0.37492752, 0.35523918]",1,2,sub-026,448000,448499.0


In [13]:
(true_pred['pred_label'] == true_pred['true_label']).value_counts()

True     41612
False    28182
Name: count, dtype: int64

In [14]:
true_pred = true_pred.rename(columns={'valore':'activation_values'})
true_pred = true_pred.rename(columns={'valore_softmax':'softmax_values'})

In [15]:
true_pred_test = true_pred[true_pred['dataset'] == 'test']

In [16]:
(true_pred_test['pred_label'] == true_pred_test['true_label']).value_counts()

True     8221
False    6433
Name: count, dtype: int64

In [17]:
true_pred.to_csv('/Volumes/Extreme SSD/progetto_tesi/improving_dementia_detection_model/explainability-dementia-alfio/local/detailed_inference/true_pred.csv', index = False)